In [706]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import dash as dash
import plotly as py
import plotly.express as px
import plotly.graph_objects as go


from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)
from dash import Dash, html, dcc
from dash import Dash, html, dcc, Input, Output

# Permanently changes the pandas settings
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

# Reset pandas settings
# pd.reset_option("all")

# TODO: implement polar chart which will look for the correct dataset name, ie "Y[YearNumber]ScoresdetDf" 
# and column names as specified below:
    # subscores index names model: 
    # 2015-16 pattern:
    # Economy_(GDP_per_Capita)
    # Family
    # Health_(Life_Expectancy)
    # Freedom
    # Trust_(Government_Corruption)
    # Generosity
    # Dystopia_Residual
# Because subscores ranges may change yearly depending on the way its calculated, we need to:
    # Normalize (0-1 rounding up) by
        # looking for local-column min and max
        # divide accordingly
    # Then dynamically feed min/max values to the polar graph axis for every actualization,
    # plus normalized values to render the radius of every go.Barpolar


In [707]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

In [708]:
# Read World Bank Data
happinessDataDf = pd.read_csv("./data/happiness-report-data.csv", encoding = "ISO-8859-1", na_values=['..', '...'])
happinessDataDf.columns = happinessDataDf.columns.str.replace(' ', '_')
display(happinessDataDf)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,"Internally_displaced_persons,_total_displaced_by_conflict_and_violence_(number_of_people)",CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent)
0,2015,YR2015,Afghanistan,AFG,-1.666416,2128.125938,0.176180,0.082786,62.659000,60.760000,...,1174000.0,2.500000,NaN,NaN,282.359,4741.431,3975.028,2.000000,2.500000,27086.07
1,2015,YR2015,Albania,ALB,2.516827,11658.894900,1.603810,0.137561,78.644000,76.404000,...,NaN,NaN,10.8,NaN,62.339,3340.637,227.354,NaN,NaN,9223.67
2,2015,YR2015,Algeria,DZA,1.646657,12071.814180,3.951984,0.327373,75.622000,74.456000,...,NaN,NaN,NaN,NaN,78833.166,55562.384,502.379,NaN,NaN,263142.90
3,2015,YR2015,Angola,AGO,-2.646420,7274.090475,1.125189,0.154684,60.655000,58.213000,...,NaN,NaN,NaN,54.0,1441.131,17080.886,0.000,NaN,NaN,87360.50
4,2015,YR2015,Argentina,ARG,1.629664,20105.198990,4.301851,0.213967,76.760000,73.294000,...,NaN,NaN,NaN,NaN,98359.941,91623.662,4008.031,NaN,NaN,370000.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,2021,YR2021,East Asia & Pacific,EAS,5.528115,20249.846280,NaN,NaN,76.393934,73.639441,...,NaN,2.678571,NaN,NaN,NaN,NaN,NaN,3.107143,2.928571,NaN
1312,2021,YR2021,Caribbean small states,CSS,4.826408,17323.612260,NaN,NaN,70.996259,68.270578,...,NaN,3.400000,NaN,NaN,NaN,NaN,NaN,3.900000,3.500000,NaN
1313,2021,YR2021,Arab World,ARB,2.014530,14793.881310,NaN,NaN,70.814483,68.832433,...,NaN,2.333333,NaN,NaN,NaN,NaN,NaN,2.083333,2.500000,NaN
1314,2021,YR2021,Africa Western and Central,AFW,1.357098,4409.450961,NaN,NaN,56.988657,55.903419,...,7726600.0,2.925000,NaN,NaN,NaN,NaN,NaN,2.900000,3.325000,NaN


In [709]:
# Merging 2015 Happiness Score
Y2015ScoresDf = pd.read_csv("./data/2015.csv")
Y2015ScoresDf.columns = Y2015ScoresDf.columns.str.replace(' ', '_')
Y2015ScoresDf.columns = Y2015ScoresDf.columns.str.replace('Country', 'Country_Name')
Y2015ScoresdetDf = Y2015ScoresDf
# display(Y2015ScoresdetDf)

Y2015ScoresDf = Y2015ScoresDf[['Country_Name',"Happiness_Score"]]
Y2015Df = pd.merge(happinessDataDf.query('Time_Code=="YR2015"'), Y2015ScoresDf, on='Country_Name')
display(Y2015Df)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
0,2015,YR2015,Afghanistan,AFG,-1.666416,2128.125938,0.176180,0.082786,62.659,60.760,...,2.5,NaN,NaN,282.359,4741.431,3975.028,2.0,2.5,27086.07,3.575
1,2015,YR2015,Albania,ALB,2.516827,11658.894900,1.603810,0.137561,78.644,76.404,...,NaN,10.8,NaN,62.339,3340.637,227.354,NaN,NaN,9223.67,4.959
2,2015,YR2015,Algeria,DZA,1.646657,12071.814180,3.951984,0.327373,75.622,74.456,...,NaN,NaN,NaN,78833.166,55562.384,502.379,NaN,NaN,263142.90,5.605
3,2015,YR2015,Angola,AGO,-2.646420,7274.090475,1.125189,0.154684,60.655,58.213,...,NaN,NaN,54.0,1441.131,17080.886,0.000,NaN,NaN,87360.50,4.033
4,2015,YR2015,Argentina,ARG,1.629664,20105.198990,4.301851,0.213967,76.760,73.294,...,NaN,NaN,NaN,98359.941,91623.662,4008.031,NaN,NaN,370000.90,6.574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2015,YR2015,Uruguay,URY,0.041680,20272.191770,1.951500,0.096265,77.483,73.392,...,NaN,18.2,NaN,95.342,6233.900,7.334,NaN,NaN,36763.09,6.485
131,2015,YR2015,Uzbekistan,UZB,5.364817,6680.184701,3.168619,0.474331,70.475,67.657,...,3.0,NaN,NaN,86016.819,7440.343,5533.503,2.0,3.5,161034.70,6.003
132,2015,YR2015,Vietnam,VNM,5.877862,7595.686738,2.393435,0.315104,73.876,69.144,...,3.5,NaN,NaN,20480.195,50963.966,83259.235,3.0,4.0,333640.40,5.360
133,2015,YR2015,Zambia,ZMB,-0.312861,3352.524885,0.305055,0.090993,61.208,58.785,...,3.0,25.2,NaN,0.000,3003.273,484.044,3.0,3.5,36308.34,5.129


In [710]:
# Merging 2016 Happiness Score
Y2016ScoresDf = pd.read_csv("./data/2016.csv")
Y2016ScoresDf.columns = Y2016ScoresDf.columns.str.replace(' ', '_')
Y2016ScoresDf.columns = Y2016ScoresDf.columns.str.replace('Country', 'Country_Name')
Y2016ScoresdetDf = Y2016ScoresDf
# display(Y2016ScoresdetDf)

Y2016ScoresDf = Y2016ScoresDf[['Country_Name',"Happiness_Score"]]
Y2016Df = pd.merge(happinessDataDf.query('Time_Code=="YR2016"'), Y2016ScoresDf, on='Country_Name')
display(Y2016Df)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
0,2016,YR2016,Afghanistan,AFG,-0.345802,2023.834656,0.153121,0.075659,63.136,61.193,...,2.5,NaN,51.7,319.029,3476.316,4829.439,2.0,2.5,26894.000,3.360
1,2016,YR2016,Albania,ALB,3.480293,12078.799330,1.556656,0.128875,78.860,76.639,...,NaN,12.7,NaN,161.348,3362.639,14.668,NaN,NaN,9087.428,4.655
2,2016,YR2016,Algeria,DZA,1.163148,11685.451420,3.840079,0.328620,75.732,74.693,...,NaN,NaN,NaN,78741.491,53519.865,0.000,NaN,NaN,265356.800,6.355
3,2016,YR2016,Angola,AGO,-6.011783,7027.146634,1.020928,0.145283,61.092,58.741,...,NaN,NaN,NaN,1496.136,17931.630,0.000,NaN,NaN,85911.420,3.866
4,2016,YR2016,Argentina,ARG,-3.110064,20307.870050,4.201802,0.206905,76.308,72.933,...,NaN,20.7,NaN,102268.963,88792.738,3010.607,NaN,NaN,373332.200,6.650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2016,YR2016,Uruguay,URY,1.363677,22522.624300,1.911174,0.084856,77.571,73.586,...,NaN,17.5,NaN,106.343,6274.237,11.001,NaN,NaN,37094.070,6.545
130,2016,YR2016,Uzbekistan,UZB,4.106073,6796.904892,3.304234,0.486138,70.755,68.016,...,3.0,NaN,NaN,75012.152,6560.263,6094.554,2.0,3.5,170891.600,5.987
131,2016,YR2016,Vietnam,VNM,5.618680,8277.684129,2.402279,0.290211,73.938,69.180,...,NaN,13.4,NaN,20384.853,52559.111,81187.380,NaN,NaN,334448.700,5.061
132,2016,YR2016,Zambia,ZMB,0.561271,3322.578918,0.316995,0.095406,61.794,59.349,...,3.0,NaN,NaN,0.000,2940.934,854.411,3.0,3.5,36392.210,4.795


In [711]:
# Merging 2017 Happiness Score
Y2017ScoresDf = pd.read_csv("./data/2017.csv")
Y2017ScoresDf = Y2017ScoresDf.replace("..", '.')
Y2017ScoresDf.columns = Y2017ScoresDf.columns.str.replace('.', '_')
Y2017ScoresdetDf = Y2017ScoresDf
# display(Y2017Df)
Y2017ScoresdetDf = Y2017ScoresdetDf.rename(columns={'Economy__GDP_per_Capita_': 'Economy_(GDP_per_Capita)'})
Y2017ScoresdetDf = Y2017ScoresdetDf.rename(columns={"Health__Life_Expectancy_": "Health_(Life_Expectancy)"})
Y2017ScoresdetDf = Y2017ScoresdetDf.rename(columns={"Freedom_to_make_life_choices": "Freedom"})
Y2017ScoresdetDf = Y2017ScoresdetDf.rename(columns={"Trust__Government_Corruption_": "Trust_(Government_Corruption)"})

# display(Y2017ScoresdetDf)

Y2017ScoresDf.columns = Y2017ScoresDf.columns.str.replace('Country', 'Country_Name')
Y2017ScoresDf = Y2017ScoresDf[['Country_Name',"Happiness_Score"]]
Y2017Df = pd.merge(happinessDataDf.query('Time_Code=="YR2017"'), Y2017ScoresDf, on='Country_Name')
# display(Y2017ScoresDf)
display(Y2017Df)

C:\Users\S\AppData\Local\Temp\ipykernel_8792\2428868660.py:4: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
0,2017,YR2017,Afghanistan,AFG,-0.253594,2096.093111,0.134187,0.064018,63.016,60.105,...,2.5,NaN,NaN,NaN,NaN,NaN,2.0,2.5,26526.930,3.794
1,2017,YR2017,Albania,ALB,3.897741,12770.987860,1.790457,0.140197,79.047,76.873,...,NaN,11.7,51.8,NaN,NaN,NaN,NaN,NaN,9656.962,4.644
2,2017,YR2017,Algeria,DZA,-0.663171,11809.483030,3.849018,0.325926,75.743,74.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,268897.100,5.872
3,2017,YR2017,Angola,AGO,-3.633446,7216.061373,0.802658,0.111232,61.680,59.331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81013.100,3.795
4,2017,YR2017,Argentina,ARG,1.757648,23597.117750,4.071397,0.172538,76.833,73.526,...,NaN,19.7,NaN,NaN,NaN,NaN,NaN,NaN,377205.700,6.599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2017,YR2017,Uruguay,URY,1.377293,23106.998190,1.783590,0.077188,77.625,73.732,...,NaN,17.1,NaN,NaN,NaN,NaN,NaN,NaN,36334.460,6.454
130,2017,YR2017,Uzbekistan,UZB,2.652485,6840.709860,3.381789,0.494362,71.010,68.257,...,3.0,NaN,NaN,NaN,NaN,NaN,2.0,3.5,176307.800,5.971
131,2017,YR2017,Vietnam,VNM,5.909238,9050.688534,2.450700,0.270775,73.963,69.260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342086.800,5.074
132,2017,YR2017,Zambia,ZMB,0.331284,3395.479686,0.393726,0.115956,62.120,59.527,...,3.0,NaN,NaN,NaN,NaN,NaN,3.0,3.5,36700.800,4.514


In [712]:
# Merging 2018 Happiness Score
Y2018ScoresDf = pd.read_csv("./data/2018.csv")
Y2018ScoresDf.columns = Y2018ScoresDf.columns.str.replace(' ', '_')
Y2018ScoresDf.columns = Y2018ScoresDf.columns.str.replace('Country_or_region', 'Country_Name')
Y2018ScoresDf.columns = Y2018ScoresDf.columns.str.replace('Score', 'Happiness_Score')
Y2018ScoresdetDf = Y2018ScoresDf
# display(Y2018Df)
Y2018ScoresdetDf.rename(columns={"GDP_per_capita": "Economy_(GDP_per_Capita)"}, inplace=True)
Y2018ScoresdetDf.rename(columns={"Social_support": "Family"}, inplace=True)
Y2018ScoresdetDf.rename(columns={"Healthy_life_expectancy": "Health_(Life_Expectancy)"}, inplace=True)
Y2018ScoresdetDf.rename(columns={"Freedom_to_make_life_choices": "Freedom"}, inplace=True)
Y2018ScoresdetDf.rename(columns={"Generosity": "Generosity"}, inplace=True)
Y2018ScoresdetDf.rename(columns={"Perceptions_of_corruption": "Trust_(Government_Corruption)"}, inplace=True)
# Y2018Df.rename(columns={"Dystopia_+_residual": "Dystopia_Residual"}, inplace=True)
# no dystopia

# display(Y2018ScoresdetDf)

Y2018ScoresDf = Y2018ScoresDf[['Country_Name',"Happiness_Score"]]
Y2018Df = pd.merge(happinessDataDf.query('Time_Code=="YR2018"'), Y2018ScoresDf, on='Country_Name')
# display(Y2018ScoresDf)
display(Y2018Df)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
0,2018,YR2018,Afghanistan,AFG,-1.688577,2109.929296,0.165367,0.078376,63.081,59.923,...,2.5,NaN,NaN,NaN,NaN,NaN,2.0,2.5,27689.580,3.632
1,2018,YR2018,Albania,ALB,4.276326,13498.196120,1.781692,0.131995,79.184,77.037,...,NaN,11.8,49.0,NaN,NaN,NaN,NaN,NaN,9453.678,4.586
2,2018,YR2018,Algeria,DZA,-0.707950,12006.010240,3.948267,0.328858,76.066,74.966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278029.900,5.295
3,2018,YR2018,Angola,AGO,-4.676622,7042.923829,0.766050,0.108769,62.144,59.582,...,NaN,22.8,NaN,NaN,NaN,NaN,NaN,NaN,75235.330,3.795
4,2018,YR2018,Argentina,ARG,-3.601610,23290.675740,3.975666,0.170698,76.999,73.631,...,NaN,20.2,NaN,NaN,NaN,NaN,NaN,NaN,375101.100,6.388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2018,YR2018,Uruguay,URY,0.335944,23738.507960,1.909548,0.080441,77.611,73.753,...,NaN,16.9,NaN,NaN,NaN,NaN,NaN,NaN,36236.510,6.379
129,2018,YR2018,Uzbekistan,UZB,3.540797,7252.137355,3.412858,0.470600,71.149,68.479,...,3.0,NaN,NaN,NaN,NaN,NaN,2.5,3.5,183812.600,6.096
130,2018,YR2018,Vietnam,VNM,6.467176,9866.217883,3.009051,0.304985,73.976,69.245,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,398894.900,5.103
131,2018,YR2018,Zambia,ZMB,0.897347,3507.795230,0.433452,0.123568,62.342,59.674,...,3.0,NaN,NaN,NaN,NaN,NaN,2.5,3.5,37562.170,4.377


In [713]:
# Merging 2019 Happiness Score
Y2019ScoresDf = pd.read_csv("./data/2019.csv")
Y2019ScoresDf.columns = Y2019ScoresDf.columns.str.replace(' ', '_')
Y2019ScoresDf.columns = Y2019ScoresDf.columns.str.replace('Country_or_region', 'Country_Name')
Y2019ScoresDf.columns = Y2019ScoresDf.columns.str.replace('Score', 'Happiness_Score')
Y2019ScoresdetDf = Y2019ScoresDf
# display(Y2019Df)
Y2019ScoresdetDf.rename(columns={"GDP_per_capita": "Economy_(GDP_per_Capita)"}, inplace=True)
Y2019ScoresdetDf.rename(columns={"Social_support": "Family"}, inplace=True)
Y2019ScoresdetDf.rename(columns={"Healthy_life_expectancy": "Health_(Life_Expectancy)"}, inplace=True)
Y2019ScoresdetDf.rename(columns={"Freedom_to_make_life_choices": "Freedom"}, inplace=True)
Y2019ScoresdetDf.rename(columns={"Generosity": "Generosity"}, inplace=True)
Y2019ScoresdetDf.rename(columns={"Perceptions_of_corruption": "Trust_(Government_Corruption)"}, inplace=True)
# Y2019Df.rename(columns={"Dystopia_+_residual": "Dystopia_Residual"}, inplace=True)
# no dystopia

# display(Y2019ScoresdetDf)

Y2019ScoresDf = Y2019ScoresDf[['Country_Name',"Happiness_Score"]]

Y2019Df = pd.merge(happinessDataDf.query('Time_Code=="YR2019"'), Y2019ScoresDf, on='Country_Name')
# display(Y2019ScoresDf)
display(Y2019Df)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
0,2019,YR2019,Afghanistan,AFG,0.932833,2167.704111,0.160850,0.074203,63.565,60.619,...,2.5,NaN,NaN,NaN,NaN,NaN,2.0,2.5,28636.500,3.203
1,2019,YR2019,Albania,ALB,2.523894,14407.460750,1.691968,0.117437,79.282,77.168,...,NaN,8.6,46.2,NaN,NaN,NaN,NaN,NaN,9008.165,4.719
2,2019,YR2019,Algeria,DZA,-0.840857,12118.004500,4.010025,0.330915,76.474,75.238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,282008.200,5.211
3,2019,YR2019,Argentina,ARG,-2.969562,23003.277180,3.740546,0.162609,77.284,73.857,...,NaN,21.2,NaN,NaN,NaN,NaN,NaN,NaN,369051.900,6.086
4,2019,YR2019,Armenia,ARM,8.208649,14921.819320,2.187334,0.146586,75.439,70.403,...,NaN,7.3,NaN,NaN,NaN,NaN,NaN,NaN,9963.242,4.559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2019,YR2019,Uruguay,URY,0.310435,24238.133050,1.893036,0.078102,77.508,73.579,...,NaN,17.4,NaN,NaN,NaN,NaN,NaN,NaN,36117.100,6.293
129,2019,YR2019,Uzbekistan,UZB,3.744518,7658.272410,3.475461,0.453818,71.344,68.782,...,3.0,NaN,NaN,NaN,NaN,NaN,2.5,3.5,188755.900,6.174
130,2019,YR2019,Vietnam,VNM,6.392604,10684.686280,3.513294,0.328816,74.093,69.369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450149.300,5.175
131,2019,YR2019,Zambia,ZMB,-1.564237,3514.688008,0.370170,0.105321,62.793,60.080,...,3.0,NaN,NaN,NaN,NaN,NaN,2.5,3.5,37570.790,4.107


In [714]:
# Merging 2020 Happiness Score
Y2020ScoresDf = pd.read_csv("./data/2020.csv")
Y2020ScoresDf.columns = Y2020ScoresDf.columns.str.replace(' ', '_')
Y2020ScoresDf.columns = Y2020ScoresDf.columns.str.replace('Country_name', 'Country_Name')
Y2020ScoresDf.columns = Y2020ScoresDf.columns.str.replace('Ladder_score', 'Happiness_Score')
Y2020ScoresdetDf = Y2020ScoresDf
# display(Y2020Df)
Y2020ScoresdetDf.rename(columns={"Logged_GDP_per_capita": "Economy_(GDP_per_Capita)"}, inplace=True)
Y2020ScoresdetDf.rename(columns={"Social_support": "Family"}, inplace=True)
Y2020ScoresdetDf.rename(columns={"Healthy_life_expectancy": "Health_(Life_Expectancy)"}, inplace=True)
Y2020ScoresdetDf.rename(columns={"Freedom_to_make_life_choices": "Freedom"}, inplace=True)
Y2020ScoresdetDf.rename(columns={"Generosity": "Generosity"}, inplace=True)
Y2020ScoresdetDf.rename(columns={"Perceptions_of_corruption": "Trust_(Government_Corruption)"}, inplace=True)
Y2020ScoresdetDf.rename(columns={"Dystopia_+_residual": "Dystopia_Residual"}, inplace=True)

# display(Y2020ScoresdetDf)

Y2020ScoresDf = Y2020ScoresDf[['Country_Name',"Happiness_Score"]]

Y2020Df = pd.merge(happinessDataDf.query('Time_Code=="YR2020"'), Y2020ScoresDf, on='Country_Name')
# display(Y2020ScoresDf)
display(Y2020Df)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
0,2020,YR2020,Afghanistan,AFG,-5.364666,2076.138380,NaN,NaN,62.575,59.866,...,2.5,NaN,49.4,NaN,NaN,NaN,2.0,2.5,NaN,2.5669
1,2020,YR2020,Albania,ALB,-2.925821,14035.354780,NaN,NaN,76.989,74.632,...,NaN,9.6,43.4,NaN,NaN,NaN,NaN,NaN,NaN,4.8827
2,2020,YR2020,Algeria,DZA,-6.729942,11438.691160,NaN,NaN,74.453,73.082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0051
3,2020,YR2020,Argentina,ARG,-10.812611,20763.288550,NaN,NaN,75.892,72.546,...,NaN,21.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.9747
4,2020,YR2020,Armenia,ARM,-6.704049,14089.239380,NaN,NaN,72.173,67.051,...,NaN,4.6,19.1,NaN,NaN,NaN,NaN,NaN,NaN,4.6768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,2020,YR2020,Uruguay,URY,-6.140011,23024.098910,NaN,NaN,78.430,74.688,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4401
127,2020,YR2020,Uzbekistan,UZB,-0.053436,7746.430969,NaN,NaN,70.331,67.876,...,3.0,NaN,NaN,NaN,NaN,NaN,2.5,3.5,NaN,6.2576
128,2020,YR2020,Vietnam,VNM,1.937355,11022.957000,NaN,NaN,75.378,70.787,...,NaN,13.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3535
129,2020,YR2020,Zambia,ZMB,-5.595733,3358.005322,NaN,NaN,62.380,59.643,...,3.0,NaN,NaN,NaN,NaN,NaN,2.5,3.5,NaN,3.7594


In [715]:
# Merging 2021 Happiness Score
Y2021ScoresDf = pd.read_csv("./data/2021.csv")
Y2021ScoresDf.columns = Y2021ScoresDf.columns.str.replace(' ', '_')
Y2021ScoresDf.columns = Y2021ScoresDf.columns.str.replace('Country_name', 'Country_Name')
Y2021ScoresDf.columns = Y2021ScoresDf.columns.str.replace('Ladder_score', 'Happiness_Score')
Y2021ScoresdetDf = Y2021ScoresDf
# display(Y2021ScoresdetDf)
Y2021ScoresdetDf.rename(columns={"Logged_GDP_per_capita": "Economy_(GDP_per_Capita)"}, inplace=True)
Y2021ScoresdetDf.rename(columns={"Social_support": "Family"}, inplace=True)
Y2021ScoresdetDf.rename(columns={"Healthy_life_expectancy": "Health_(Life_Expectancy)"}, inplace=True)
Y2021ScoresdetDf.rename(columns={"Freedom_to_make_life_choices": "Freedom"}, inplace=True)
Y2021ScoresdetDf.rename(columns={"Generosity": "Generosity"}, inplace=True)
Y2021ScoresdetDf.rename(columns={"Perceptions_of_corruption": "Trust_(Government_Corruption)"}, inplace=True)
Y2021ScoresdetDf.rename(columns={"Dystopia_+_residual": "Dystopia_Residual"}, inplace=True)

# display(Y2021ScoresdetDf)

Y2021ScoresDf = Y2021ScoresDf[['Country_Name',"Happiness_Score"]]
Y2021Df = pd.merge(happinessDataDf.query('Time_Code=="YR2021"'), Y2021ScoresDf, on='Country_Name')
# display(Y2021ScoresDf)
display(Y2021Df)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
0,2021,YR2021,Afghanistan,AFG,-22.965296,1665.805842,NaN,NaN,61.982,58.915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.523
1,2021,YR2021,Albania,ALB,9.526027,15814.411570,NaN,NaN,76.463,74.087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.117
2,2021,YR2021,Algeria,DZA,1.798420,12128.274990,NaN,NaN,76.377,74.879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.887
3,2021,YR2021,Argentina,ARG,9.357176,23649.670300,NaN,NaN,75.390,72.182,...,NaN,18.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.929
4,2021,YR2021,Armenia,ARM,6.254220,15592.487360,NaN,NaN,72.043,66.554,...,NaN,6.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2021,YR2021,Uruguay,URY,4.453092,25048.723970,NaN,NaN,75.436,71.654,...,NaN,17.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.431
125,2021,YR2021,Uzbekistan,UZB,5.318632,8497.448308,NaN,NaN,70.862,68.334,...,3.5,NaN,NaN,NaN,NaN,NaN,3.0,3.5,NaN,6.179
126,2021,YR2021,Vietnam,VNM,1.699390,11676.108560,NaN,NaN,73.618,69.117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.411
127,2021,YR2021,Zambia,ZMB,1.669097,3555.920229,NaN,NaN,61.223,58.487,...,3.0,NaN,NaN,NaN,NaN,NaN,2.5,3.5,NaN,4.073


In [716]:
# merging all data
dataArray = [Y2015Df, Y2016Df,Y2017Df,Y2018Df,Y2019Df,Y2020Df,Y2021Df]
mergedData = pd.concat(dataArray)
# display(mergedData)

In [717]:
# create sample chart from data
fig = px.scatter(mergedData, x="Life_expectancy_at_birth,_total_(years)", y="GDP_per_capita,_PPP_(current_international_$)", animation_frame="Time", hover_name="Country_Name" )
fig.show()

In [718]:
# create sample geospatial visualization from data
fig1 = px.choropleth(mergedData, 
                    locations="Country_Code",
                    hover_name="Country_Name",
                     color="Happiness_Score",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig1.show()

In [719]:
geospatialVisual = html.Div([
                        dcc.Graph(
                            id='geospatial',
                        )
                    ], style={'display': 'inline-block', 'padding': '0 20'})

In [720]:
happinessScoreVisual = html.Div([
                            dcc.Graph(
                                id='happiness-score-scatter-line'
                            )
                        ], style={'display': 'inline-block', 'padding': '0 20'})

In [721]:
app.layout = html.Div([
    html.Div([
        geospatialVisual,
#         happinessScoreVisual,
    ], style={
        'width': '50%',
        'padding': '10px 5px'
    }),
#     html.Div([
#         dcc.Graph(id='variable1-scatter-line'),
#         dcc.Graph(id='variable2-scatter-line'),

    html.Div(dcc.Slider(
        min=mergedData['Time'].min(),
        max=mergedData['Time'].max(),
        id='year-slider',
        value=mergedData['Time'].max(),
        marks={str(time): str(time) for time in mergedData['Time'].unique()}
    ), style={'width': '97%', 'padding': '10 20 20 20'}),
])

In [722]:
@app.callback(
    Output('geospatial', 'figure'),
    Input('year-slider', 'value'))
def update_graph(year_value):
    df=mergedData[mergedData['Time'] == year_value]
    fig =  px.choropleth(df, 
                    locations="Country_Code",
                    hover_name="Country_Name",
                    color="Happiness_Score",
                    color_continuous_scale=px.colors.sequential.Plasma)

    return fig

In [723]:
# happiness score visualization
@app.callback(
    Output('happiness-score-scatter-line', 'figure'),
    Input('geospatial', 'hoverData'))
def update_graph(year_value):
    countryZone = hoverData['points'][0]['customdata']
    df=mergedData[mergedData['Country_Zone'] == countryZone]
    fig =  px.scatter(mergedData, 
                      x="Life_expectancy_at_birth,_total_(years)", 
                      y="GDP_per_capita,_PPP_(current_international_$)",
                      hover_name="Country_Name" )

    return fig

In [724]:
# @app.callback(
#     Output('hover-data', 'children'),
#     Input('geospatial', 'hoverData'))
# def display_hover_data(hoverData):
#     return json.dumps(hoverData, indent=2)
# if __name__ == '__main__':
#     app.run_server(port=8088, debug=True)

In [725]:
## WIP from there and below !!

values = mergedData[(mergedData['Country_Name'] == "Finland")
                  & (mergedData['Time'] == 2021)]

display(values)

,Time,Time_Code,Country_Name,Country_Code,GDP_per_capita_growth_(annual_%),"GDP_per_capita,_PPP_(current_international_$)",CO2_emissions_(metric_tons_per_capita),CO2_emissions_(kg_per_PPP_$_of_GDP),"Life_expectancy_at_birth,_total_(years)","Life_expectancy_at_birth,_male_(years)",...,CPIA_quality_of_public_administration_rating_(1=low_to_6=high),Proportion_of_people_living_below_50_percent_of_median_income_(%),Multidimensional_poverty_headcount_ratio_(%_of_total_population),CO2_emissions_from_gaseous_fuel_consumption_(kt),CO2_emissions_from_liquid_fuel_consumption_(kt),CO2_emissions_from_solid_fuel_consumption_(kt),"CPIA_transparency,_accountability,_and_corruption_in_the_public_sector_rating_(1=low_to_6=high)",CPIA_policy_and_institutions_for_environmental_sustainability_rating_(1=low_to_6=high),Total_greenhouse_gas_emissions_(kt_of_CO2_equivalent),Happiness_Score
36,2021,YR2021,Finland,FIN,2.759999,55217.28456,NaN,NaN,81.934146,79.3,...,NaN,NaN,14.2,NaN,NaN,NaN,NaN,NaN,NaN,7.842


In [726]:
# # Data for the polar chart
# categories = ['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5', 'Category 6', 'Category 7']
# values = [4, 7, 2, 5, 1, 6, 3]
# colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta']

# # Create traces for each category
# traces = []
# for i in range(len(categories)):
#     trace = go.Barpolar(
#         r=[values[i]],
#         theta=[categories[i]],
#         width=1,
#         marker=dict(color=colors[i]),
#         name=categories[i]
#     )
#     traces.append(trace)

# # Layout for the polar chart
# layout = go.Layout(
#     polar=dict(
#         radialaxis=dict(range=[0, max(values)]),
#     ),
# )

# # Create the figure
# fig2 = go.Figure(data=traces, layout=layout)
# fig2.show()


# I don't understand how callbacks works, but I wanted to show the detail of each subscore 
# of the happiness score once user hovers over a point of the happiness score visualization, 
# i.e: once a couple Country/Year is chosen, display the score detail following previously commented code

# happiness score detail visualization
# @app.callback(
#     Output('happiness-score-detail-polarchart', 'figure'),
#     Input('happiness-score-scatter-line', 'hoverData'))
# def update_graph(geoHoverData):
#     countryName = geoHoverData['points'][0]['hovertext']
#     df=mergedData[mergedData['Country_Name'] == countryName]
#     fig =  px.scatter(df, 
#                       x="Time", 
#                       y="Happiness_Score",
#                       hover_name="Country_Name" )
#     fig.update_traces(mode='lines+markers')
#     return fig


In [727]:

# Data for the polar chart
# categories = ['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5', 'Category 6', 'Category 7']
categories = ['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5', 'Category 6', 'Category 7']
values = [4, 7, 2, 5, 1, 6, 3]
# values = mergedData[(mergedData['Country_Name'] == countryName)
#                   & (mergedData['Time'] == year_value)]

colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta']

# countryName = geoHoverData['points'][0]['hovertext']
#     df=

# Create traces for each category
traces = []
for i in range(len(categories)):
    trace = go.Barpolar(
        r=[values[i]],
        theta=[categories[i]],
        width=1,
        marker=dict(color=colors[i]),
        name=categories[i]
    )
    traces.append(trace)

# Layout for the polar chart
layout = go.Layout(
    polar=dict(
        radialaxis=dict(range=[0, max(values)]),
    ),
)

# Create the figure
fig2 = go.Figure(data=traces, layout=layout)
fig2.show()


if __name__ == '__main__':
    app.run_server(port=8088, debug=True)